In [1]:
import os
import copy
import wget
import time
import asyncio
import warnings
import logging

import numpy as np

from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS

%matplotlib inline

In [2]:
attcs = ATTCS()
cam = salobj.Remote(attcs.domain, "GenericCamera", 1)

await asyncio.gather(attcs.start_task, cam.start_task) 

RemoteTelemetry(ATDome, 0, position) falling behind; read 23 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 10 messages


[[None, None, None, None, None, None, None], None]

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 12 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 12 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 12 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 13 messages


In [59]:
# Declare target name
target_name='tetCol'
expTime=60

In [60]:
reset_offset = {'m1': 0.0,
          'm2': 0.0,
          'x': 0. ,
          'y': 0.,
          'z': 0.,
          'u': 0.0,
          'v': 0.0
          }



In [63]:
dz = 0.8

offset_intra = {'m1': 0.0,
          'm2': 0.0,
          'x': 0.0,
          'y': 0.0,
          'z': -dz,
          'u': 0.0,
          'v': 0.0
          }

offset_extra = {'m1': 0.0,
          'm2': 0.0,
          'x': 0.0,
          'y': 0.0,
          'z': 2*dz,
          'u': 0.0,
          'v': 0.0
          }


try:
    await cam.cmd_stopLiveView.start()
except salobj.AckError:
    print("ack error")

# Intrafocal hexapod setting - offsets -0.5mm from central focus value above

print('Move to intrafocal position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                              z=hex_Z-dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(**offset_intra)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print('Take intrafocal image')
cam.evt_endReadout.flush()

try:
    await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_intra')
except Exception as e:
    await attcs.ataos.cmd_offset.set_start(z=dz)
    raise e

intra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

# Extrafocal hexapod setting
print('Move to extra-focal hexapod position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                                      z=hex_Z+dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(**offset_extra)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

# r.evt_endReadout.flush()
print('Take extra-focal image')
cam.evt_endReadout.flush()
try:
    await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_extra')
except Exception as e:
    await attcs.ataos.cmd_offset.set_start(z=-dz)
    raise e

extra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()

print(f"============")
print(f"dz={dz}")
print(intra_endron.imageName)
print(extra_endron.imageName)
print(f"============")

try:
    await cam.cmd_startLiveView.set_start(expTime=0.5)
except salobj.AckError:
    print("ack error")

attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(z=-dz)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print(f"['{intra_endron.imageName}.fits'], ['{extra_endron.imageName}.fits']")
print(np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle))

# intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
# extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'
# intra_fname = wget.download(intra_wget_url)
# print(f"============")
# print(f'{intra_fname}')
# extra_fname = wget.download(extra_wget_url)
# print(f'{extra_fname}')
# print(f"============")




Move to intrafocal position
Take intrafocal image
Move to extra-focal hexapod position
Take extra-focal image
dz=0.8
1580103907-tetCol_intra-0-1
1580103974-tetCol_extra-0-1
['1580103907-tetCol_intra-0-1.fits'], ['1580103974-tetCol_extra-0-1.fits']
-61.446525050286574


In [64]:
intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'

intra_fname = wget.download(intra_wget_url, out="cwfs/data")
extra_fname = wget.download(extra_wget_url, out="cwfs/data")

print(intra_fname,extra_fname)

cwfs/data/1580103907-tetCol_intra-0-1.fits cwfs/data/1580103974-tetCol_extra-0-1.fits


RemoteEvent(ATMCS, 0, target) falling behind; read 18 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 11 messages


In [15]:
offset = {'m1': 0.0,
          'm2': 0.0,
          'x': -0.35151226,
          'y': -0.02652471,
          'z':  0.01910018,
          'u': 0.0,
          'v': 0.0
          }


await attcs.ataos.cmd_offset.set_start(**offset)
attcs.ataos.cmd_offset.set(**reset_offset)

# await attcs.ataos.cmd_applyAxisOffset.set_start(axis="x", offset=x_offset)
# await attcs.ataos.cmd_applyAxisOffset.set_start(axis="y", offset=y_offset)

True

In [35]:
await cam.cmd_stopLiveView.start()

cam.evt_endReadout.flush()

await cam.cmd_takeImages.set_start(numImages=1, expTime=5, shutter=True, imageSequenceName=target_name+'_test')

img1_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

offset = {'m1': 0.0,
          'm2': 0.0,
          'x': -0.5,
          'y': 0.,
          'z':  0.0,
          'u': 0.0,
          'v': 0.0
          }


attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(**offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

await asyncio.sleep(10)

await cam.cmd_takeImages.set_start(numImages=1, expTime=5, shutter=True, imageSequenceName=target_name+'_test')

img2_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

offset = {'m1': 0.0,
          'm2': 0.0,
          'x': 0.5,
          'y': 0.,
          'z':  0.0,
          'u': 0.0,
          'v': 0.0
          }


attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(**offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)



In [28]:
await cam.cmd_takeImages.set_start(numImages=1, expTime=5, shutter=True, imageSequenceName=target_name+'_test')

img2_endron = await cam.evt_endReadout.next(flush=False, timeout=5)



In [36]:
print(img1_endron.imageName)
print(img2_endron.imageName)

1580101959-tetCol_test-0-1
1580101977-tetCol_test-0-1


In [37]:
img1_wget_url = 'http://139.229.170.216:8000/data/'+img1_endron.imageName+'.fits'
img2_wget_url = 'http://139.229.170.216:8000/data/'+img2_endron.imageName+'.fits'

img1_fname = wget.download(img1_wget_url, out="cwfs/data")
img2_fname = wget.download(img2_wget_url, out="cwfs/data")

print(img1_fname,img2_fname)

cwfs/data/1580101959-tetCol_test-0-1.fits cwfs/data/1580101977-tetCol_test-0-1.fits


In [38]:
await cam.cmd_startLiveView.set_start(expTime=0.5)

In [26]:
attcs.ataos.cmd_offset.set(**reset_offset)
await attcs.ataos.cmd_offset.set_start(z=-dz)
attcs.ataos.cmd_offset.set(**reset_offset)

True

In [33]:
offset = {'m1': 0.0,
          'm2': 0.0,
          'x': 0.5,
          'y': 0.,
          'z':  0.0,
          'u': 0.0,
          'v': 0.0
          }


attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(**offset)
attcs.ataos.cmd_offset.set(**reset_offset)


True

In [49]:
# await cam.cmd_stopLiveView.start()

cam.evt_endReadout.flush()

await cam.cmd_takeImages.set_start(numImages=1, expTime=5, shutter=True, imageSequenceName=target_name+'_test2')

img1_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

attcs.atmcs.evt_allAxesInPosition.flush()
await attcs.atptg.cmd_offsetAzEl.set_start(az=30+5., el=15)
try:
    await attcs.atmcs.evt_allAxesInPosition.next(flush=False, timeout=10.)
except Exception:
    pass

await cam.cmd_takeImages.set_start(numImages=1, expTime=5, shutter=True, imageSequenceName=target_name+'_test2')

img2_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

await attcs.atptg.cmd_offsetAzEl.set_start(az=5., el=15)



In [48]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=5., el=15)

In [50]:
img1_wget_url = 'http://139.229.170.216:8000/data/'+img1_endron.imageName+'.fits'
img2_wget_url = 'http://139.229.170.216:8000/data/'+img2_endron.imageName+'.fits'

img1_fname = wget.download(img1_wget_url, out="cwfs/data")
img2_fname = wget.download(img2_wget_url, out="cwfs/data")

print(img1_fname,img2_fname)

cwfs/data/1580102749-tetCol_test2-0-1.fits cwfs/data/1580102771-tetCol_test2-0-1.fits


In [70]:
await cam.cmd_startLiveView.set_start(expTime=.5)

In [55]:


offset = {'m1': 0.0,
          'm2': 0.0,
          'x': -0.5,
          'y': 0.,
          'z':  0.0,
          'u': 0.0,
          'v': 0.0
          }

attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(**offset)
attcs.ataos.cmd_offset.set(**reset_offset)



True

In [56]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=5., el=15)

In [67]:
await cam.cmd_stopLiveView.start()

In [68]:
for exp in [0.5, 1, 3, 5, 15]:
    await cam.cmd_takeImages.set_start(numImages=1, expTime=5, shutter=True, imageSequenceName="Trap")

RemoteEvent(ATMCS, 0, target) falling behind; read 60 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 12 messages


In [69]:
for exp in [0.5, 1, 3, 5, 15]:
    endron = await cam.evt_endReadout.next(flush=False, timeout=5)
    wget_url = 'http://139.229.170.216:8000/data/'+endron.imageName+'.fits'
    fname = wget.download(img1_wget_url, out="cwfs/data")
    print(fname)






cwfs/data/1580102749-tetCol_test2-0-1 (1).fits
cwfs/data/1580102749-tetCol_test2-0-1 (1).fits
cwfs/data/1580102749-tetCol_test2-0-1 (1).fits
cwfs/data/1580102749-tetCol_test2-0-1 (1).fits
cwfs/data/1580102749-tetCol_test2-0-1 (1).fits
